In [1]:
import tensorflow as tf
import numpy as np
import time
import sys
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math
import cv2
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_util
from PIL import Image

In [2]:
w = 60#360
h = 80#178
ws = 150
hs = 49
box  = [
    [ws,hs+1353, w, h],
    [ws,hs+1531, w, h],
    [ws,hs+1711, w, h],
#     [ws,hs+1888, w, h],
    
    [ws+361,hs+1353, w, h],
    [ws+361,hs+1531, w, h],
    [ws+361,hs+1711, w, h],
    [ws+361,hs+1888, w, h],
    
    [ws+721,hs+1353, w, h],
    [ws+721,hs+1531, w, h],
    [ws+721,hs+1711, w, h],
#     [ws+721,hs+1888, w, h],
]

def get_image(filename):
    image = tf.image.decode_jpeg(tf.read_file(filename), channels=1)
    with tf.Session() as sess:
        d = image.eval()
        ss = d.reshape(-1, w*h)
#         print(ss.shape)
#         img_string = ' '.join(str(s) for s in ss[0])
    return ss

def getNumImg(name):
    img = Image.open(name)
    data_list = []
    for i in range(len(box)):
        b = (box[i][0], box[i][1], box[i][0]+box[i][2], box[i][1]+box[i][3])
        roi = img.crop(b)
#         print(type(roi))
        roi.save('tmp.png')
        d = get_image('tmp.png')
        d = d/255.0
#         tf.cast(d, tf.float32)
        ss = d.reshape(w*h,1)
#         image_val = tf.concat(0,ss)
        data_list.append(ss)
    X = np.vstack(data_list)
    X = X.reshape(-1, w*h, 1)
    
    return X

In [3]:
def getNum(fileName):
    
    test_list = getNumImg(fileName)
    name_str = [
        '数字0',
        '数字1',
        '数字2',
        '数字3',
        '数字4',
        '数字5',
        '数字6',
        '数字7',
        '数字8',
        '数字9',
        ]
    num_label = []
    with tf.gfile.FastGFile('./num.pb', 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
    with tf.Session() as sess:
    #     with open('./num.pb','rb') as f:
    #         graph_def = tf.GraphDef()
    #         graph_def.ParseFromString(f.read()) 
        out_tensor_name = 'layer/out/p/output:0'
        out_data_tensor = sess.graph.get_tensor_by_name(out_tensor_name)


        batch_xs = test_list[0:10,...]
        batch_xs = batch_xs.reshape(-1, w*h)

        data_tensor = sess.run(out_data_tensor,{'input/x:0':batch_xs,'input/k:0':1.0})
    #         tf.cast(batch_xs, tf.float64)
    #     output = tf.import_graph_def(graph_def, input_map={'input/x:0':batch_xs, 'input/k':1.0}, return_elements=['layer/out/p/output:0'], name='a')
    #         output = tf.import_graph_def(graph_def, input_map={'layer/p/dense:0':batch_xs, 'input/k':1.0}, return_elements=['layer/out/p/output:0'], name='a') 
    #     out = sess.run(output)
    #     print(out)
    #     print(data_tensor)
        for i in range(10):
            predictions = np.squeeze(data_tensor[i])
            top_k = predictions.argsort()[-1:][::-1]
            for node_id in top_k:     
                score = predictions[node_id]
                num_label.append(node_id)
        #         print(score)
                print('位置 %s %s (识别概率：%.5f)' % (i,name_str[int(node_id)], score))
#             print()
    return num_label

In [4]:
num = getNum('images/003.png')
print(num)

位置 0 数字8 (识别概率：0.93569)
位置 1 数字1 (识别概率：0.99989)
位置 2 数字0 (识别概率：0.99853)
位置 3 数字6 (识别概率：0.99994)
位置 4 数字7 (识别概率：0.99987)
位置 5 数字9 (识别概率：0.99979)
位置 6 数字2 (识别概率：0.93746)
位置 7 数字5 (识别概率：0.99293)
位置 8 数字4 (识别概率：0.99955)
位置 9 数字3 (识别概率：0.98886)
[8, 1, 0, 6, 7, 9, 2, 5, 4, 3]


In [5]:
num = getNum('images/015.png')
print(num)

位置 0 数字4 (识别概率：0.99983)
位置 1 数字8 (识别概率：0.97521)
位置 2 数字3 (识别概率：0.98526)
位置 3 数字0 (识别概率：0.99030)
位置 4 数字9 (识别概率：0.99791)
位置 5 数字2 (识别概率：0.99923)
位置 6 数字7 (识别概率：0.99950)
位置 7 数字5 (识别概率：0.99293)
位置 8 数字1 (识别概率：0.99994)
位置 9 数字6 (识别概率：0.99994)
[4, 8, 3, 0, 9, 2, 7, 5, 1, 6]


In [ ]:
位置 0 数字8 (识别概率：0.99684)
位置 1 数字1 (识别概率：0.99997)
位置 2 数字9 (识别概率：0.82950)
位置 3 数字6 (识别概率：0.54548)
位置 4 数字7 (识别概率：0.99937)
位置 5 数字9 (识别概率：0.99996)
位置 6 数字2 (识别概率：0.33494)
位置 7 数字5 (识别概率：0.96579)
位置 8 数字4 (识别概率：0.99874)
位置 9 数字3 (识别概率：0.99987)
[8, 1, 9, 6, 7, 9, 2, 5, 4, 3]

位置 0 数字4 (识别概率：0.99677)
位置 1 数字6 (识别概率：0.72714)
位置 2 数字3 (识别概率：0.99553)
位置 3 数字9 (识别概率：0.77154)
位置 4 数字9 (识别概率：0.96354)
位置 5 数字2 (识别概率：0.99974)
位置 6 数字7 (识别概率：0.99316)
位置 7 数字5 (识别概率：0.96579)
位置 8 数字1 (识别概率：0.99993)
位置 9 数字6 (识别概率：0.54596)
[4, 6, 3, 9, 9, 2, 7, 5, 1, 6]